In [1]:
from skiier import *
from ski_env import *
import matplotlib.pyplot as plt
import gym
import papermill as pm
%matplotlib inline

/home/ai2-yoda/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Hyper Parameters
LEARNING_RATE = 0.001
GAMMA = 0.9
MEMORY_CAPACITY = 25000

In [3]:
ski_env = gym.make("Skiing-v0")
ski_env_w = Env_wrapper(ski_env)
skiier = Skiier(action_space = 3, learning_rate = LEARNING_RATE, LAMBDA = 0.5, 
                GAMMA = GAMMA, MEMORY_CAPACITY = MEMORY_CAPACITY, BATCH_SIZE = 512)

/home/ai2-yoda/gym/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


In [4]:
num_steps = 10000
num_episodes = 100 #converges around 30-45 with current hyper parameters

for episode in range(num_episodes):
    state = ski_env_w.reset()
    
    for time_step in range(num_steps):
        action = skiier.act(state)
        #print("Action:", action)
        next_state, reward, done, info = ski_env_w.step(action)
        skiier.observe(state, action, reward, next_state, done)
        state = next_state
        skiier.episode = episode
        if done:
            print("episode: {}/{}, score: {}, final_epsilon: {}".format(episode, num_episodes, time_step, skiier.epsilon))
            pm.record("episode", episode)
            pm.record("time_step", time_step)
            pm.record("epsilon", skiier.epsilon)
            break
            
    skiier.replay()

episode: 0/1, score: 1898, final_epsilon: 1


# Outputs

In [5]:
def play_game(num_steps):
    state = ski_env_w.reset()
    state_list = [state]
    reward_list = []
    for time_step in range(num_steps):
        action = skiier.act(state)
        next_state, reward, done, info = ski_env_w.step(action)
        skiier.observe(state, action, reward, next_state, done)
        state = next_state
        state_list.append(state)
        reward_list.append(reward)
        skiier.episode = episode
        if done:
            break
    return state_list, reward_list

In [6]:
#states_out, rewards_out = play_game(num_steps = 10000)

In [13]:
file_params = "-".join([str(LEARNING_RATE), str(GAMMA), str(MEMORY_CAPACITY)]).replace(".","dot")
file_name = "models/model_HP_{}_at_{}.hdf5".format(file_params, time_step)
skiier.brain.model.save(file_name)